 03_parks.ipynb — Parks & Greenspace processing

Purpose
- Inspect and prepare parks (and later) canopy data used to compute parks-related metrics per each FSA.
- This notebook contains the focused, heavier computations (area intersections, proximity sampling, canopy clipping and integration) as well as the tail-end of the massive debugging that took place in 03_parks.ipynb, in order to keep it cleaner. 
- The first cells are lightweight checks to: confirm file paths, CRSs, and a small sample of attributes so it can safely run the heavier steps below.

Notes
- This notebook writes updates to the file (data/processed/fsa_master.geojson) when it runs the integration cell.

In [ ]:
# Imports and lightweight configuration for parks processing
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# Project paths (adjust if your repo layout differs)
ROOT = Path.cwd().parent
DATA_RAW = ROOT / "data" / "raw"
DATA_PROC = ROOT / "data" / "processed"

FSA_MASTER = DATA_PROC / "fsa_master.geojson"                # processed FSAs (master file) 
PARKS_FILE = DATA_RAW / "parks" / "espace_vert.geojson"     # parks source 
CANOPY_DIR = DATA_RAW / "canopy"                            # canopy source
CANOPY_CLIP = DATA_PROC / "canopy_montreal.geojson"         # clipped canopy (produced later)

# FSA code standardization function
def normalize_fsa_col(gdf, preferred_cols=("FSA_CODE","CFSAUID","FSA")):
    for c in preferred_cols:
        if c in gdf.columns:
            gdf["FSA_CODE"] = gdf[c].astype(str).str.strip().str.upper()
            return gdf
    # fallback to index
    gdf["FSA_CODE"] = gdf.index.astype(str)
    return gdf

In [ ]:
# Quick lightweight checks: existence, CRS, counts, sample rows. 
# This includes much AI help to weed through large files without loading them fully, which was an issue considering natinoal canopy data.

def quick_preview():
    print("Root folder:", ROOT)
    print("Processed FSA file:", FSA_MASTER.exists(), FSA_MASTER)
    print("Parks file:", PARKS_FILE.exists(), PARKS_FILE)
    print("Canopy dir exists:", CANOPY_DIR.exists(), CANOPY_DIR)
    if CANOPY_CLIP.exists():
        print("Clipped canopy (exists):", CANOPY_CLIP)
    print()

    # 1) FSA master (processed) - load only attributes and first few rows
    if FSA_MASTER.exists():
        print("Loading FSA master (light)...")
        fsa = gpd.read_file(FSA_MASTER)
        fsa = normalize_fsa_col(fsa)
        print(" FSA rows:", len(fsa))
        print(" FSA CRS:", fsa.crs)
        print(" FSA columns:", list(fsa.columns))
        # show a small sample of the fields we care about if present
        sample_cols = [c for c in ["FSA_CODE","population","crime_count","crime_rate_per_1000",
                                   "parks_area_per_sqkm","parks_mean_distance_m","canopy_fraction","final_score"] if c in fsa.columns]
        print("\nSample FSA rows (first 6):")
        display(fsa[sample_cols].head(6))
    else:
        print("FSA master not found; run upstream pipeline to produce:", FSA_MASTER)

    # 2) Parks file - small polygon layer, ok to read summary
    if PARKS_FILE.exists():
        print("\nLoading parks (light):")
        try:
            parks = gpd.read_file(PARKS_FILE)
            print(" Parks features:", len(parks))
            print(" Parks CRS:", parks.crs)
            print(" Geometry types:", parks.geometry.type.value_counts().to_dict())
            # show name-like columns if any
            name_cols = [c for c in ["Nom","Name","nom","name","NOM","NAME"] if c in parks.columns]
            if name_cols:
                print(" Parks sample names (first 6):")
                display(parks[name_cols].head(6))
        except Exception as e:
            print(" Failed to read parks file (error):", e)
    else:
        print("\nParks file not present at:", PARKS_FILE)

    # 3) Canopy source diagnostics (do NOT read full canopy if large)
    print("\nCanopy diagnostics:")
    if CANOPY_DIR.exists():
        # find vector candidates (quick)
        vecs = list(Path(CANOPY_DIR).glob("*.geojson")) + list(Path(CANOPY_DIR).glob("*.json")) + list(Path(CANOPY_DIR).glob("*.shp"))
        rasts = list(Path(CANOPY_DIR).glob("*.tif")) + list(Path(CANOPY_DIR).glob("*.tiff"))
        if vecs:
            canopy_path = vecs[0]
            print(" Canopy vector found:", canopy_path)
            # read only first record if possible
            try:
                sample = gpd.read_file(canopy_path, rows=1)
                print("  Declared CRS:", sample.crs, "| sample geom type:", sample.geometry.type.iloc[0])
            except TypeError:
                # fallback to fiona for metadata
                with fiona.open(str(canopy_path)) as src:
                    print("  Declared CRS (from fiona):", src.crs)
                    print("  First feature bounds:", src.bounds)
        elif rasts:
            print(" Canopy raster found (not loaded here):", rasts[0])
            print("  Note: raster processing requires rasterio; skip heavy raster ops in this lightweight cell.")
        else:
            print(" No canopy files found in", CANOPY_DIR)
    else:
        print(" Canopy directory not found:", CANOPY_DIR)

    print("\nDone. If all file paths and CRSs look correct you can proceed to the parks computations below this cell.")

# Run the preview
quick_preview()

Root folder: /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas
Processed FSA file: True /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/processed/fsa_master.geojson
Parks file: True /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/raw/parks/espace_vert.geojson
Canopy dir exists: True /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/raw/canopy
Clipped canopy (exists): /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/processed/canopy_montreal.geojson

Loading FSA master (light)...
 FSA rows: 97
 FSA CRS: EPSG:4326
 FSA columns: ['CFSAUID', 'PRUID', 'PRNAME', 'FSA_CODE', 'crime_count', 'parks_count_x', 'crime_score', 'parks_score', 'final_score', 'population', 'crime_rate_per_1000', 'fsa_area_km2', 'parks_area_km2', 'parks_area_per_sqkm', 'transit_route_km', 'transit_km_per_sqkm', 'transit_score', 'parks_mean_distance_m', 'parks_area_score', 'parks_prox_score', 'par

,FSA_CODE,population,crime_count,crime_rate_per_1000,parks_area_per_sqkm,parks_mean_distance_m,canopy_fraction,final_score
0,H1A,32516.0,4194.0,128.982655,0.196037,330.062415,0.258829,0.508001
1,H3Z,11813.0,1621.0,137.221705,0.000000,689.563239,0.259985,0.589005
2,H3Y,9747.0,1009.0,103.519031,0.003620,827.157922,0.483580,0.540609
3,H1B,20160.0,3028.0,150.198413,0.019228,1216.911447,0.124562,0.398255
4,H1J,10308.0,2447.0,237.388436,0.182674,454.097173,0.109654,0.413276
5,H1K,34821.0,4185.0,120.186095,0.127171,127.447866,0.231851,0.504408



Loading parks (light):
 Parks features: 2242
 Parks CRS: EPSG:4326
 Geometry types: {'Polygon': 2233, 'MultiPolygon': 9}
 Parks sample names (first 6):


,Nom
0,Université Concordia - Campus Loyola
1,Boisé
2,Maurice-Cullen
3,Décarie / Zone Tampon #2
4,Décarie / Zone Tampon #2
5,Trenholme



Canopy diagnostics:
 Canopy vector found: /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/raw/canopy/canopee-2019.geojson
  Declared CRS: EPSG:4326 | sample geom type: MultiPolygon

Done. If all file paths and CRSs look correct you can proceed to the parks computations below this cell.


In [4]:
# Recompute parks_score using canopy (heavy canopy weight) and update fsa_master.geojson
# Paste as one cell in 03_parks.ipynb (run after canopy_montreal.geojson is created)
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

proj_root = Path.cwd().parent
FSA_IN = proj_root / "data" / "processed" / "fsa_master.geojson"
CANOPY_CLIP = proj_root / "data" / "processed" / "canopy_montreal.geojson"
OUT = FSA_IN

# ---- configuration: heavy canopy weighting (edit here if needed) ----
w_canopy = 0.50   # canopy weight (dominant)
w_area = 0.30     # park area weight
w_prox = 0.20     # park proximity weight
# ---------------------------------------------------------------------

# safety checks
if not FSA_IN.exists():
    raise FileNotFoundError(f"FSA master not found: {FSA_IN}")
print("Loading FSA:", FSA_IN)
g = gpd.read_file(FSA_IN)
# ensure canonical FSA_CODE
if "FSA_CODE" not in g.columns:
    if "CFSAUID" in g.columns:
        g["FSA_CODE"] = g["CFSAUID"].astype(str).str.strip().str.upper()
    elif "FSA" in g.columns:
        g["FSA_CODE"] = g["FSA"].astype(str).str.strip().str.upper()
    else:
        g["FSA_CODE"] = g.index.astype(str)
g["FSA_CODE"] = g["FSA_CODE"].str.upper()

# metric copy for intersections/areas
g_m = g.to_crs(epsg=3857).copy()
g_m["fsa_area_km2"] = g_m.geometry.area / 1e6

# load clipped canopy and compute canopy area per FSA (if present)
canopy_area_km2 = {}
if CANOPY_CLIP.exists():
    print("Loading clipped canopy:", CANOPY_CLIP)
    canopy = gpd.read_file(CANOPY_CLIP).to_crs(epsg=3857)
    canopy_poly = canopy[canopy.geometry.type.isin(["Polygon","MultiPolygon"])].copy()
    print("Canopy polygons (loaded):", len(canopy_poly))
    if len(canopy_poly) > 0:
        # spatial join canopy -> FSA and compute intersection area (m2)
        try:
            sj = gpd.sjoin(canopy_poly, g_m[["FSA_CODE","geometry"]], how="inner", predicate="intersects")
        except Exception:
            sj = gpd.sjoin(canopy_poly, g_m[["FSA_CODE","geometry"]], how="inner", op='intersects')
        if not sj.empty:
            merged = sj.merge(g_m[["FSA_CODE","geometry"]], on="FSA_CODE", suffixes=("_can","_fsa"))
            pa = {}
            for idx, row in merged.iterrows():
                try:
                    inter = row.geometry_can.intersection(row.geometry_fsa)
                    if inter is not None and not inter.is_empty:
                        pa[row.FSA_CODE] = pa.get(row.FSA_CODE, 0.0) + inter.area
                except Exception:
                    pass
            canopy_area_km2 = {k: v/1e6 for k,v in pa.items()}
else:
    print("Clipped canopy not found; canopy_fraction will be zero. Expected path:", CANOPY_CLIP)

# assign canopy fields into metric DF
g_m["canopy_area_km2"] = g_m["FSA_CODE"].map(canopy_area_km2).fillna(0.0)
g_m["canopy_fraction"] = g_m.apply(lambda r: (r["canopy_area_km2"] / r["fsa_area_km2"]) if r["fsa_area_km2"]>0 else 0.0, axis=1)

# canopy_score normalization (min-max)
if g_m["canopy_fraction"].max() != g_m["canopy_fraction"].min():
    g_m["canopy_score"] = (g_m["canopy_fraction"] - g_m["canopy_fraction"].min()) / (g_m["canopy_fraction"].max() - g_m["canopy_fraction"].min())
else:
    g_m["canopy_score"] = 0.0

# ensure parks area/prox scores exist (fallbacks)
if "parks_area_score" not in g_m.columns:
    if "parks_area_per_sqkm" in g_m.columns:
        s = g_m["parks_area_per_sqkm"].fillna(0.0).astype(float)
        g_m["parks_area_score"] = (s - s.min()) / (s.max() - s.min()) if s.max()!=s.min() else 0.0
    else:
        g_m["parks_area_score"] = 0.0

if "parks_prox_score" not in g_m.columns:
    if "parks_mean_distance_m" in g_m.columns:
        prox = g_m["parks_mean_distance_m"].fillna(g_m["parks_mean_distance_m"].max(skipna=True))
        g_m["parks_prox_score"] = 1.0 - ((prox - prox.min()) / (prox.max() - prox.min())) if prox.max()!=prox.min() else 0.0
    else:
        g_m["parks_prox_score"] = 0.0

# combine into parks_score using configured weights
g_m["parks_score"] = (w_canopy * g_m["canopy_score"].fillna(0.0) +
                      w_area * g_m["parks_area_score"].fillna(0.0) +
                      w_prox * g_m["parks_prox_score"].fillna(0.0))

# recompute final_score as mean of available components
components = [c for c in ["crime_score","parks_score","transit_score"] if c in g_m.columns]
if components:
    g_m["final_score"] = g_m[components].fillna(0.0).mean(axis=1)
else:
    g_m["final_score"] = g_m.get("final_score", 0.0)

# write back to GeoJSON (WGS84)
g_out = g_m.to_crs(epsg=4326)
g_out.to_file(OUT, driver="GeoJSON")
print("Wrote updated master GeoJSON to:", OUT)
print("FSAs with canopy_fraction > 0:", int((g_out['canopy_fraction']>0).sum()), "/", len(g_out))
print("Canopy fraction range:", float(g_out['canopy_fraction'].min()), "->", float(g_out['canopy_fraction'].max()))

# sample rows for quick review (H9R / H9S)
try:
    display(g_out[g_out["FSA_CODE"].isin(["H9R","H9S"])][["FSA_CODE","parks_area_per_sqkm","parks_mean_distance_m","canopy_fraction","canopy_area_km2","parks_score","final_score"]])
except Exception:
    pass

Loading FSA: /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/processed/fsa_master.geojson
Loading clipped canopy: /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/processed/canopy_montreal.geojson
Canopy polygons (loaded): 514124
Wrote updated master GeoJSON to: /Users/tylercoull/Desktop/(ConU)/ConU F25/GEOG 464/NewFinal/RentAtlas/data/processed/fsa_master.geojson
FSAs with canopy_fraction > 0: 97 / 97
Canopy fraction range: 0.06824648616846461 -> 0.5620946926593101


,FSA_CODE,parks_area_per_sqkm,parks_mean_distance_m,canopy_fraction,canopy_area_km2,parks_score,final_score
79,H9R,0.000000,4041.265190,0.240337,7.400029,0.225803,0.416643
80,H9S,0.000029,4018.725681,0.336321,6.437993,0.271426,0.497551
